In [1]:
# import dependencies
import os
import io
import bs4
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
# from selenium import webdriver
# from lxml import html
import csv
from datetime import datetime
from pytz import timezone
import pandas as pd

In [2]:
# url of ucsc college 9 & 10 dining hall
url = 'https://nutrition.sa.ucsc.edu/nutframe.asp?sName=UC+Santa+Cruz+Dining&locationNum=40&locationName=Colleges+Nine+%26+Ten+Dining+Hall&naFlag=1'

In [3]:
# scrap the menu data from the webpage
# since the menu data is cannot be scrapped from the html file directly
# we have to make additional requests to get the frame page contents 
with requests.Session() as session:
    response = session.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    for frame in soup.select("frameset frame"):
        frame_url = urljoin(url, frame["src"])
        response = session.get(frame_url)
        frame_soup = BeautifulSoup(response.content, 'html.parser') 
        # print(frame_soup.prettify())

In [4]:
# extract both the meal name and food name from html
meal_name = frame_soup.find_all('div', attrs={'class': 'menusampmeals'})
food_name = frame_soup.find_all('div', attrs={'class': 'menusamprecipes'})

In [5]:
# convert the name of the meal type in html form to a list form
meal_type = [item.string for item in meal_name]
print(meal_type)
print('Length: ', len(meal_type))

['Breakfast', 'Lunch', 'Dinner']
Length:  3


In [6]:
# convert the food name in html form to a list form
food_list = [item.string for item in food_name]
print(food_list)
print('Length: ', len(food_list))

['Banana and Coconut Pancakes', 'Cage Free Scrambled Eggs', 'Chicken Mango Sausage', 'Coconut Syrup', 'Hard-boiled Cage Free Eggs', 'Natural BridgesTofu Scramble', 'Oatmeal Gluten-Free', 'Roasted Yukon Gold Potatoes', 'Steamed Rice', 'Chicken Tortilla Soup', 'Oven Roasted Allergen Free Chicken Thigh', 'Stuffed Shells in classic Marinara Sauce', 'Cheesy Garlic Bread Sticks', 'Steamed Seasonal Vegetables', 'Armenian Salad', 'BAR Middle Eastern', 'Beef and Lamb Gyro', 'Falafels', 'Lime Cilantro Basmati Rice', 'Tahini Sauce', 'Tzatziki Sauce', "Chef's Special", 'Oven Roasted Allergen Free Chicken Thigh', 'Cheese Pizza', 'Cilantro Flatbread with Red Onions and Mushrooms', 'BBQ Shredded Pork', 'Chicken Tenders', 'Condiments', 'Mac & Cheese', 'Mac and Cheese Bar', 'Roasted Vegetables', 'Vegan Macaroni & Cheese']
Length:  32


In [7]:
# In case of dupliate food name, the list should be converted to a set
# sort
food_list = sorted(list(set(food_list)))
print(food_list)
print('Sorted length: ', len(food_list))

['Armenian Salad', 'BAR Middle Eastern', 'BBQ Shredded Pork', 'Banana and Coconut Pancakes', 'Beef and Lamb Gyro', 'Cage Free Scrambled Eggs', 'Cheese Pizza', 'Cheesy Garlic Bread Sticks', "Chef's Special", 'Chicken Mango Sausage', 'Chicken Tenders', 'Chicken Tortilla Soup', 'Cilantro Flatbread with Red Onions and Mushrooms', 'Coconut Syrup', 'Condiments', 'Falafels', 'Hard-boiled Cage Free Eggs', 'Lime Cilantro Basmati Rice', 'Mac & Cheese', 'Mac and Cheese Bar', 'Natural BridgesTofu Scramble', 'Oatmeal Gluten-Free', 'Oven Roasted Allergen Free Chicken Thigh', 'Roasted Vegetables', 'Roasted Yukon Gold Potatoes', 'Steamed Rice', 'Steamed Seasonal Vegetables', 'Stuffed Shells in classic Marinara Sauce', 'Tahini Sauce', 'Tzatziki Sauce', 'Vegan Macaroni & Cheese']
Sorted length:  31


In [11]:
# preference list, labeled by myself
# pref_1 = [1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0 ]
# pref_2 = [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0]
# pref_3 = [0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1]
# pref_4 = [0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0]
# pref_5 = [1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0]
pref_6 = []
for i in food_list:
    print(i)
    preference = int(input())
    pref_6.append(preference)
print(pref_6)
print('Preference length: ', len(pref_6))

Armenian Salad
0
BAR Middle Eastern
0
BBQ Shredded Pork
1
Banana and Coconut Pancakes
1
Beef and Lamb Gyro
0
Cage Free Scrambled Eggs
1
Cheese Pizza
1
Cheesy Garlic Bread Sticks
1
Chef's Special
0
Chicken Mango Sausage
1
Chicken Tenders
1
Chicken Tortilla Soup
0
Cilantro Flatbread with Red Onions and Mushrooms
0
Coconut Syrup
0
Condiments
0
Falafels
0
Hard-boiled Cage Free Eggs
0
Lime Cilantro Basmati Rice
0
Mac & Cheese
1
Mac and Cheese Bar
1
Natural BridgesTofu Scramble
0
Oatmeal Gluten-Free
0
Oven Roasted Allergen Free Chicken Thigh
0
Roasted Vegetables
0
Roasted Yukon Gold Potatoes
1
Steamed Rice
1
Steamed Seasonal Vegetables
0
Stuffed Shells in classic Marinara Sauce
0
Tahini Sauce
0
Tzatziki Sauce
0
Vegan Macaroni & Cheese
0
[0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
Preference length:  31


In [13]:
# timezone setting
us_pacific = timezone('US/Pacific')
time = datetime.now(us_pacific)
us_time = time.strftime('%Y-%m-%d')
print('Current US/Pacific time: ', us_time)

Current US/Pacific time:  2018-09-15


In [14]:
# write the food list to a csv file
path = '/Users/7w0r4ng3s/Desktop/menu_scraping/data/{}.csv'.format(us_time)
print('Current path: ', path)
path_2 = '/Users/7w0r4ng3s/Desktop/menu_scraping/data/'

Current path:  /Users/7w0r4ng3s/Desktop/menu_scraping/data/2018-09-15.csv


In [15]:
def write_data():
    with open(path, "w") as output:
        writer = csv.writer(output, lineterminator='\n')
        for key, val in zip(food_list, pref_6):
            writer.writerow([key, val])
    print('write_csv: COMPLETED')
    # add index and column names
    df = pd.read_csv(path, names=['food', 'pref'])
    df.index.names = ['index']
    df.to_csv(path)
    print('add_column_name: COMPLETED')
    
write_data()

write_csv: COMPLETED
add_column_name: COMPLETED


In [16]:
# def merge_data():
#     # TODO: Figure out a way to get rid of the unnamed: 0 column
#     # TODO: Modify merge_data() so that new data can be append to data.csv
#     files = [f for f in os.listdir('.') if os.path.isfile(f)]

#     merged = []

#     for f in files:
#         filename, ext = os.path.splitext(f)
#         if ext == '.csv':
#             read = pd.read_csv(f)
#             merged.append(read)

#     result = pd.concat(merged)
#     result.to_csv('data.csv')
    
# merge_data()

In [17]:
def append_data():
    df1 = pd.read_csv('Data.csv', index_col='index')
    df1.index.names = ['index']
    df2 = pd.read_csv(path, index_col='index')
    df3 = df1.append(df2).reset_index()
    df3 = df3.drop('index', 1).sort_values('food').reset_index().drop('index', 1)
    df3.index.names = ['index']
    df3.to_csv('Data.csv')
    
append_data()